In [3]:
#!git clone https://github.com/fayrose/EgyptianTranslation.git
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [8]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 12.9 MB/s 
     |████████████████████████████████| 15.8 MB 135 kB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 1.2 MB 41.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [6]:
string = """
# egy_eng.yaml

## Where the samples will be written
save_data: EgyptianTranslation/onmt-files/run/example
## Where the vocab(s) will be written
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: EgyptianTranslation/compiled_corpora/aligned_train.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_train.eng.csv
    valid:
        path_src: EgyptianTranslation/compiled_corpora/aligned_val.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_val.eng.csv
"""
with open("EgyptianTranslation/egy_eng.yaml", 'w') as yaml_file:
  yaml_file.write(string)

In [9]:
!onmt_build_vocab -config EgyptianTranslation/egy_eng.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-20 03:14:24,754 INFO] Counter vocab from -1 samples.
[2021-09-20 03:14:24,754 INFO] n_sample=-1: Build vocab on full datasets.
[2021-09-20 03:14:24,764 INFO] corpus_1's transforms: TransformPipe()
[2021-09-20 03:14:25,867 INFO] Counters src:3183
[2021-09-20 03:14:25,868 INFO] Counters tgt:6356
[2021-09-20 03:14:26,234 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.egy exists, may overwrite...
[2021-09-20 03:14:26,721 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.eng exists, may overwrite...


In [10]:
string = """
# Vocabulary files that were just created
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng

# Train on a single GPU
world_size: 1

# Where to save the checkpoints
save_model: EgyptianTranslation/onmt-files/run/model
save_checkpoint_steps: 1000
train_steps: 3000
valid_steps: 1500

# Optimization
model_dtype: "fp32"
optim: "adam"
learning_rate: 2
warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
param_init: 0
param_init_glorot: true
label_smoothing: 0.1
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
heads: 8
rnn_size: 512
word_vec_size: 512
max_generator_batches: 2
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
accum_count: 4
attention_dropout: [0.1]
enc_layers: 6
dec_layers: 6
gpu_ranks: 0
"""

with open("EgyptianTranslation/egy_eng.yaml", 'a') as yaml_file:
  yaml_file.write(string)

In [11]:
!onmt_train --config EgyptianTranslation/egy_eng.yaml

[2021-09-20 03:14:39,956 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-09-20 03:14:39,957 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-20 03:14:39,959 INFO] Missing transforms field for valid data, set to default: [].
[2021-09-20 03:14:39,960 INFO] Parsed 2 corpora from -data.
[2021-09-20 03:14:39,961 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-09-20 03:14:39,961 INFO] Loading vocab from text file...
[2021-09-20 03:14:39,961 INFO] Loading src vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.egy
[2021-09-20 03:14:39,972 INFO] Loaded src vocab has 3183 tokens.
[2021-09-20 03:14:39,974 INFO] Loading tgt vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.eng
[2021-09-20 03:14:39,992 INFO] Loaded tgt vocab has 6356 tokens.
[2021-09-20 03:14:39,995 INFO] Building fields with vocab in counters...
[2021-09-20 03:14:40,003 INFO]  * tgt vocab size: 6360.
[2021-

In [40]:
!onmt_translate -model EgyptianTranslation/onmt-files/run/model_step_3000.pt -src /content/drive/MyDrive/EgyptianTranslation/compiled_corpora/aligned_test.egy.csv -output EgyptianTranslation/onmt-files/run/pred_3000.txt

[2021-09-20 04:10:02,001 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2021-09-20 04:10:44,674 INFO] PRED AVG SCORE: -0.9037, PRED PPL: 2.4687


In [41]:
ref_path = "EgyptianTranslation/compiled_corpora/aligned_test.eng.csv"
with open(ref_path, 'r') as ref_file:
  ref = [list(map(lambda x: x.strip('\n'), ref_file.readlines()))]

print(len(ref[0]))
ref[0][-10:]

660


['and the fields were flooded three palms high with the liquid',
 'by the might of the majesty of this god',
 'then this goddess came in the early morning',
 'and found that these fields were flooded',
 'and her gaze was pleased by it',
 'then she drank and it pleased her heart',
 'after she returned drunk without having perceived mankind',
 'the majesty of re said to this goddess',
 'welcome in peace o gracious one',
 'thus beautiful women came into being in momemphis']

In [42]:
pred_path = "EgyptianTranslation/onmt-files/run/pred_3000.txt"
with open(pred_path, 'r') as pred_file:
  pred = list(map(lambda x: x.strip('\n'), pred_file.readlines()))

print(len(pred))
pred[-10:]

660


['then there was very many fields after the water',
 'as the might of the majesty of this god',
 'then this official who is in the morning',
 'she found a fan',
 'and it is good out there',
 'then there was done on account of the goodness',
 'when she came out without a plummet',
 'then the majesty of Re said to this city',
 'when it is born in peace',
 'how good it is a master of tent']

In [18]:
!pip install sacrebleu

     |████████████████████████████████| 90 kB 5.6 MB/s 


In [43]:
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction

split_ref = [[item] for item in ref[0]]
split_pred = [item for item in pred]
bleu_score.corpus_bleu(split_ref, split_pred, smoothing_function=SmoothingFunction().method5) * 100

41.68753829829091

In [44]:
from sacrebleu.metrics import BLEU

bleu = BLEU()
split_ref = [[item] for item in ref[0]]
split_pred = [item for item in pred]
bleu.corpus_score(split_pred, split_ref)

BLEU = 33.57 80.0/44.4/25.0/14.3 (BP = 1.000 ratio = 1.000 hyp_len = 10 ref_len = 10)